In [5]:
!pip list | grep pystan

pystan                             3.3.0


In [8]:
import stan
import nest_asyncio
nest_asyncio.apply()

schools_code = """
data {
  int<lower=0> J;         // number of schools
  array[J] real y;              // estimated treatment effects
  array[J] real<lower=0> sigma; // standard error of effect estimates
}
parameters {
  real mu;                // population treatment effect
  real<lower=0> tau;      // standard deviation in treatment effects
  vector[J] eta;          // unscaled deviation from mu by school
}
transformed parameters {
  vector[J] theta = mu + tau * eta;        // school treatment effects
}
model {
  mu ~ normal(0, 2);   // Dummy prior for mu
  tau ~ cauchy(0, 2);   // Dummy prior for tau

  target += normal_lpdf(eta | 0, 1);       // prior log-density
  target += normal_lpdf(y | theta, sigma); // log-likelihood
}
"""

schools_data = {"J": 8,
                "y": [28,  8, -3,  7, -1,  1, 18, 12],
                "sigma": [15, 10, 16, 11,  9, 11, 10, 18]}

posterior = stan.build(schools_code, data=schools_data)
fit = posterior.sample(num_chains=4, num_samples=1000)
eta = fit["eta"]  # array with shape (8, 4000)
df = fit.to_frame()  # pandas `DataFrame`

Building...

INFO:aiohttp.access:127.0.0.1 [02/Apr/2024:13:47:22 +0000] "POST /v1/models/lzs7qnq7/params HTTP/1.1" 404 249 "-" "Python/3.7 aiohttp/3.8.6"
INFO:httpstan:Building model-specific services extension module for `models/lzs7qnq7`.
INFO:aiohttp.access:127.0.0.1 [02/Apr/2024:13:47:22 +0000] "POST /v1/models HTTP/1.1" 201 234 "-" "Python/3.7 aiohttp/3.8.6"



Building: 28.3s, done.ERROR:aiohttp.server:Error handling request
Traceback (most recent call last):
  File "/home/springnuance/miniconda3/envs/cc_env_hitl/lib/python3.7/site-packages/aiohttp/web_protocol.py", line 433, in _handle_request
    resp = await request_handler(request)
  File "/home/springnuance/miniconda3/envs/cc_env_hitl/lib/python3.7/site-packages/aiohttp/web_app.py", line 504, in _handle
    resp = await handler(request)
  File "/home/springnuance/miniconda3/envs/cc_env_hitl/lib/python3.7/site-packages/httpstan/views.py", line 253, in handle_show_params
    services_module = httpstan.models.import_services_extension_module(model_name)
  File "/home/springnuance/miniconda3/envs/cc_env_hitl/lib/python3.7/site-packages/httpstan/models.py", line 90, in import_services_extension_module
    module: ModuleType = importlib.util.module_from_spec(spec)  # type: ignore
  File "<frozen importlib._bootstrap>", line 583, in module_from_spec
  File "<frozen importlib._bootstrap_extern

ValueError: The JSON document has an improper structure: missing or superfluous commas, braces, missing keys, etc.